# 1. Introdução

Neste trabalho foi analisado o conjunto de dados disponível em https://archive.ics.uci.edu/ml/datasets/Parkinsons, contendo métricas retiradas da voz de pacientes com doença de Parkinson e de pacientes saudáveis. Para mais detalhes sobre essas métricas confira no link do *dataset*.

O estudo a seguir tem como objetivo a construção do melhor classificador para discriminar entre pacientes com doença de Parkinson e pacientes saudáveis, utilizando como atributos as métricas disponíveis no conjunto de dados. Para avaliar os classificadores a metodologia adotada fora realizar 100 (cem) rodadas independentes de separação do conjunto de dados entre treino/teste e posteriormente calculada a acurácia dos classificadores no conjunto de teste. Devido ao tamanho reduzido do dataset, 195 amostras, a razão treino/teste escolhida foi de 70%/30%, propiciando um conjunto de teste de aproximadamente 58 amostras.

O código abaixo foi utilizado para carregar o conjunto de dados no sistema computacional e averiguar o número de amostras e atributos.

In [1]:
# carregando dataset
import pandas as pd

dataset = pd.read_csv("./parkinsons_data.csv")

temp = dataset.columns.values.tolist()
features = [e for e in temp if e not in ('name', 'status')] # retirando colunas 'name' e 'status'
features = dataset[features]

targets = dataset['status'].copy()

print('Número de amostras: {}.\nNúmero de atributos: {}.'.format(targets.shape[0],features.shape[1]))

Número de amostras: 195.
Número de atributos: 22.


# 2. K vizinhos mais próximos (k-NN)

Para a avaliação do classificador k-vizinhos mais próximos foram utilizadas as funções da biblioteca *scikit-learn*. Uma busca em grade de dois hiper-parâmetros também foi realizada:
 - Tipo de mudança de escala dos dados: se 'min-max', que deixa os atributos entre 0 e 1, ou a 'std', conhecida como normalização estatística, deixando os atributos com média zero e variância unitária;
 - O valor de '$k$', ou quantidade de vizinhos mais próximos. A busca foi realizada entre os valores de 1 a 10, inclusivo.

In [2]:
# Função para mudar a escala dos dados
def scale_feat(X_train, X_test, scaleType='min-max'):
    if scaleType=='min-max' or scaleType=='std':
        X_tr_norm = np.copy(X_train) # fazendo cópia para deixar original disponível
        X_ts_norm = np.copy(X_test)
        scaler = MinMaxScaler() if scaleType=='min-max' else StandardScaler()
        scaler.fit(X_tr_norm)
        X_tr_norm = scaler.transform(X_tr_norm)
        X_ts_norm = scaler.transform(X_ts_norm)
        return (X_tr_norm, X_ts_norm)
    else:
        raise ValueError("Tipo de escala não definida. Use 'min-max' ou 'std'.")

In [3]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import numpy as np

In [4]:
%%time
# cabeçalho da tabela de resultados
cabecalho = ['Média', 'Mediana', 'Mínimo', 'Máximo', 'Desv. Padrão', 'Sensib. média', 'Especif. média']

scales = ['min-max', 'std']   # tipos de escalonamentos possíveis
ks = [i for i in range(1,11)] # valores possíveis de k para k-NN
nn_data = np.zeros((len(ks)*len(scales), len(cabecalho))) # matriz que guardará resultados numéricos
for scale in scales:
    for k in ks:
        especificidade = 0
        sensibilidade  = 0
        acc = [0]*100
        for i in range(100):
            # divisão treino/teste 
            X_train, X_test, y_train, y_test = train_test_split(features.values, 
                                                                targets.values,
                                                                test_size=0.3)
            # escalonar os dados
            X_tr_norm, X_ts_norm = scale_feat(X_train, X_test, scaleType=scale)

            # construindo classificador
            k_nn = KNeighborsClassifier(n_neighbors=k, n_jobs=-1)
            k_nn.fit(X_tr_norm, y_train)

            # calculando as métricas de avaliação
            cm = confusion_matrix(y_test, k_nn.predict(X_ts_norm))
            total=sum(sum(cm))

            acc[i] = (cm[0,0]+cm[1,1])/total
            especificidade += cm[0,0]/(cm[0,0]+cm[0,1])
            sensibilidade  += cm[1,1]/(cm[1,1]+cm[1,0])

        especificidade/=100 # Valores médios
        sensibilidade/=100

        index = k-1 if scale=='min-max' else k+9 # indíce da linha dos dados
        nn_data[index,:] = np.matrix([np.mean(acc), np.median(acc), min(acc), max(acc), 
                             np.std(acc), sensibilidade, especificidade])

idx_label      = ['{}-NN [\'min-max\']'.format(k) for k in ks]
idx_label.extend(['{}-NN [\'std\']'.format(k) for k in ks])
df_knn = pd.DataFrame(nn_data, columns=cabecalho, index=[idx_label])

CPU times: user 26.9 s, sys: 3.48 s, total: 30.3 s
Wall time: 3min 39s


In [5]:
df_knn

,Média,Mediana,Mínimo,Máximo,Desv. Padrão,Sensib. média,Especif. média
1-NN ['min-max'],0.938644,0.949153,0.864407,1.000000,0.031140,0.954675,0.893021
2-NN ['min-max'],0.906102,0.915254,0.830508,0.983051,0.035262,0.892517,0.951056
3-NN ['min-max'],0.924068,0.932203,0.813559,0.983051,0.030927,0.950540,0.846765
4-NN ['min-max'],0.913559,0.915254,0.796610,1.000000,0.036470,0.926652,0.878727
5-NN ['min-max'],0.911525,0.915254,0.779661,1.000000,0.040079,0.960724,0.768435
6-NN ['min-max'],0.900000,0.898305,0.796610,0.983051,0.041204,0.935857,0.797161
7-NN ['min-max'],0.891525,0.898305,0.745763,0.966102,0.042067,0.962489,0.685443
8-NN ['min-max'],0.892034,0.898305,0.779661,0.966102,0.040654,0.941851,0.756559
9-NN ['min-max'],0.872881,0.881356,0.745763,0.983051,0.046756,0.958007,0.618352
10-NN ['min-max'],0.858136,0.864407,0.728814,0.949153,0.045331,0.940077,0.626888


Podemos ver, na tabela, que o melhor desempenho ocorreu quando $k=1$ e a mudança de escala é do tipo *'min-max'*, sendo então essa configuração adotada no futuro comparativo entre os classificadores.

In [6]:
df_knn.sort_values('Média', ascending=False).iloc[0,:].to_frame().transpose()

,Média,Mediana,Mínimo,Máximo,Desv. Padrão,Sensib. média,Especif. média
1-NN ['min-max'],0.938644,0.949153,0.864407,1.0,0.03114,0.954675,0.893021


# 3. Regressão linear

Uma regressão linear simples através de mínimos quadrados foi realizada e, antes de classificar o conjunto de teste, a seguinte função foi adicionada à saída do modelo:

$$
\phi(u) = \left\{
        \begin{array}{ll}
            1 & \quad u \geq 0,5 \\
            0 & \quad u < 0,5
        \end{array}
    \right.
$$

In [7]:
%%time
from sklearn import linear_model

# matriz que guardará os resultados numéricos
reg_data = np.zeros((len(scales), len(cabecalho))) 

for j in range(len(scales)):
    acc = [0]*100
    especificidade = 0
    sensibilidade  = 0
    for i in range(100):
        # divisão treino/teste 
        X_train, X_test, y_train, y_test = train_test_split(features.values, 
                                                            targets.values,
                                                            test_size=0.3)
        # escalonar os dados
        X_tr_norm, X_ts_norm = scale_feat(X_train, X_test, scaleType=scales[j])
        
        # construindo o classificador
        reg = linear_model.LinearRegression(n_jobs=-1)
        reg.fit(X_tr_norm, y_train)

        # calculando as métricas de avaliação
        cm = confusion_matrix(y_test, reg.predict(X_ts_norm)>=.5)
        total=sum(sum(cm))

        acc[i] = (cm[0,0]+cm[1,1])/total
        especificidade += cm[0,0]/(cm[0,0]+cm[0,1])
        sensibilidade  += cm[1,1]/(cm[1,1]+cm[1,0])

    especificidade/=100 # Valores médios
    sensibilidade /=100
    
    reg_data[j,:] = np.matrix([np.mean(acc), np.median(acc), min(acc), max(acc), 
                         np.std(acc), sensibilidade, especificidade])

index = ['Reg. Linear [\'{}\']'.format(scale) for scale in scales]
reg_df = pd.DataFrame(reg_data, columns=cabecalho, index=index)

CPU times: user 334 ms, sys: 0 ns, total: 334 ms
Wall time: 677 ms


In [8]:
reg_df

,Média,Mediana,Mínimo,Máximo,Desv. Padrão,Sensib. média,Especif. média
Reg. Linear ['min-max'],0.863898,0.864407,0.762712,0.966102,0.04237,0.944628,0.622252
Reg. Linear ['std'],0.860339,0.864407,0.762712,0.966102,0.03644,0.946452,0.596918


O classificador com normalização estatística foi adotado por apresentar melhor desempenho, mesmo que a diferença seja tão pequena que torne quase arbitrária a escolha entre os dois.

In [9]:
reg_df.sort_values('Média', ascending=False).iloc[0,:].to_frame().transpose()

,Média,Mediana,Mínimo,Máximo,Desv. Padrão,Sensib. média,Especif. média
Reg. Linear ['min-max'],0.863898,0.864407,0.762712,0.966102,0.04237,0.944628,0.622252


# 4. Regressão logística

Para a regressão logística também foi realizada uma busca em grade pelos hiper-parâmetros:
- Tipo de normalização dos dados, *'min-max'* ou *'std'*;
- Tipo da norma da função custo, $l_1$ ou $l_2$;
- Constante $C$, que é o inverso da força da regularização aplicada à função custo.

In [10]:
%%time
from sklearn.linear_model import LogisticRegression

# Hiper-parâmetros
scales = ['min-max', 'std']
penalties = ['l1', 'l2']
Cs = [10**(i) for i in range(-3,5)]

nCases = len(scales)*len(penalties)*len(Cs) # número de combinações dos hiperparâmetros
log_data = np.zeros((nCases,len(cabecalho)))
index = ['string']*nCases # lista de índices a serem salvos

count = 0
for scale in scales:
    for penalty in penalties:
        for C in Cs:       
            acc = [0]*100
            especificidade = 0
            sensibilidade  = 0
            for i in range(100):
                # divisão treino/teste 
                X_train, X_test, y_train, y_test = train_test_split(features.values, 
                                                                    targets.values,
                                                                    test_size=0.3)
                # escalonar os dados
                X_tr_norm, X_ts_norm = scale_feat(X_train, X_test, scaleType=scale)
                
                solver = 'lbfgs' if scale=='l2' else 'liblinear'
                clf = LogisticRegression(penalty=penalty, C=C, solver=solver
                                         ,max_iter=int(1e4)
                                        )
                clf.fit(X_tr_norm, y_train)

                cm = confusion_matrix(y_test, clf.predict(X_ts_norm)>.5)
                total=sum(sum(cm))

                acc[i] = (cm[0,0]+cm[1,1])/total
                especificidade += cm[0,0]/(cm[0,0]+cm[0,1])
                sensibilidade  += cm[1,1]/(cm[1,1]+cm[1,0])

            especificidade/=100 # Valores médios
            sensibilidade /=100

            log_data[count,:] = np.matrix([np.mean(acc), np.median(acc), min(acc), max(acc), 
                                 np.std(acc), sensibilidade, especificidade])
            index[count] = 'Reg. log. [{} / {} / {}]'.format(scale, penalty, C)
            
            count+=1
                
log_df = pd.DataFrame(log_data, columns=cabecalho, index=[index])

/home/romulo/Dropbox/Mestrado/[00 old]/[DISCIPLINA] Reconhecimento de Padrões/Trabalhos Computacionais/TC1/venv/lib/python3.6/site-packages/sklearn/svm/base.py:931: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


CPU times: user 1min 6s, sys: 0 ns, total: 1min 6s
Wall time: 1min 6s


In [11]:
log_df

,Média,Mediana,Mínimo,Máximo,Desv. Padrão,Sensib. média,Especif. média
Reg. log. [min-max / l1 / 0.001],0.253898,0.254237,0.152542,0.389831,0.045478,0.000000,1.000000
Reg. log. [min-max / l1 / 0.01],0.246780,0.254237,0.135593,0.355932,0.043893,0.000000,1.000000
Reg. log. [min-max / l1 / 0.1],0.757288,0.762712,0.644068,0.898305,0.051570,1.000000,0.000714
Reg. log. [min-max / l1 / 1],0.842712,0.847458,0.728814,0.932203,0.039319,0.959848,0.505007
Reg. log. [min-max / l1 / 10],0.839153,0.847458,0.728814,0.915254,0.039857,0.917818,0.592924
Reg. log. [min-max / l1 / 100],0.844746,0.847458,0.694915,0.932203,0.042116,0.900825,0.677588
Reg. log. [min-max / l1 / 1000],0.842881,0.847458,0.694915,0.949153,0.047690,0.901929,0.665896
Reg. log. [min-max / l1 / 10000],0.838136,0.830508,0.728814,0.932203,0.043436,0.892638,0.675281
Reg. log. [min-max / l2 / 0.001],0.750000,0.754237,0.644068,0.847458,0.049028,1.000000,0.000000
Reg. log. [min-max / l2 / 0.01],0.754407,0.762712,0.593220,0.847458,0.050479,1.000000,0.000000


In [12]:
log_df.sort_values('Média', ascending=False).head()

,Média,Mediana,Mínimo,Máximo,Desv. Padrão,Sensib. média,Especif. média
Reg. log. [std / l2 / 10],0.855254,0.864407,0.711864,0.932203,0.039707,0.920512,0.657771
Reg. log. [std / l2 / 100],0.853559,0.847458,0.728814,0.932203,0.037787,0.914396,0.670653
Reg. log. [std / l1 / 100],0.851525,0.847458,0.711864,0.949153,0.042618,0.901609,0.698101
Reg. log. [std / l1 / 10],0.850508,0.847458,0.745763,0.949153,0.045755,0.906930,0.674034
Reg. log. [min-max / l2 / 1000],0.849661,0.847458,0.711864,0.932203,0.044628,0.910171,0.663283


Apesar da pequena diferença entre os primeiros colocados, em relação à média de acertos, o primeiro colocado, destacado na tabela abaixo, foi escolhido como ponto de referência para a comparação com os outros classificadores.

In [13]:
log_df.sort_values('Média', ascending=False).iloc[0,:].to_frame().transpose()

,Média,Mediana,Mínimo,Máximo,Desv. Padrão,Sensib. média,Especif. média
Reg. log. [std / l2 / 10],0.855254,0.864407,0.711864,0.932203,0.039707,0.920512,0.657771


# 5. Distância mínima ao centróide (DMC)

Para o classificador distância mínima ao centróide só foi realizado um teste para saber qual tipo de normalização dos dados era mais vantajosa.

In [14]:
%%time
from numpy.linalg import norm

scales = ['min-max', 'std']
nCases = len(scales) # número de combinações dos hiperparâmetros
DMC_data = np.zeros((nCases,len(cabecalho)))
index = ['string']*nCases # lista de índices a serem salvos

count = 0
for scale in scales:
    acc = [0]*100
    especificidade = 0
    sensibilidade  = 0
    for i in range(100):
        # divisão treino/teste 
        X_train, X_test, y_train, y_test = train_test_split(features.values, 
                                                            targets.values,
                                                            test_size=0.3)
        # escalonar os dados
        X_tr_norm, X_ts_norm = scale_feat(X_train, X_test, scaleType=scale)
        
        # Cálculo dos centróides
        c_parkinson = np.matmul(X_tr_norm.T, (y_train==1)) / sum(y_train)
        c_saudavel  = np.matmul(X_tr_norm.T, (y_train==0)) / (len(y_train) - sum(y_train))
        
        # Predição do conjunto de teste
        y_pred = [1 if norm(u-c_parkinson) < norm(u-c_saudavel) else 0 for u in X_ts_norm]
        
        # métricas de avaliação
        cm = confusion_matrix(y_test, y_pred)
        total=sum(sum(cm))

        acc[i] = (cm[0,0]+cm[1,1])/total
        especificidade += cm[0,0]/(cm[0,0]+cm[0,1])
        sensibilidade  += cm[1,1]/(cm[1,1]+cm[1,0])

    especificidade/=100 # Valores médios
    sensibilidade /=100

    DMC_data[count,:] = np.matrix([np.mean(acc), np.median(acc), min(acc), max(acc), 
                                     np.std(acc), sensibilidade, especificidade])
    index[count] = 'DMC [\'{}\']'.format(scale)

    count+=1

CPU times: user 300 ms, sys: 0 ns, total: 300 ms
Wall time: 300 ms


In [15]:
DMC_df = pd.DataFrame(DMC_data, columns=cabecalho, index=[index])
DMC_df

,Média,Mediana,Mínimo,Máximo,Desv. Padrão,Sensib. média,Especif. média
DMC ['min-max'],0.756271,0.762712,0.627119,0.847458,0.044119,0.730282,0.832183
DMC ['std'],0.732712,0.745763,0.627119,0.864407,0.051735,0.683050,0.884309


O classificador Distância Mínima ao Centróide com normalização mínimo-máximo foi escolhido por apresentar uma média de acertos maior.

In [16]:
DMC_df.sort_values('Média', ascending=False).iloc[0,:].to_frame().transpose()

,Média,Mediana,Mínimo,Máximo,Desv. Padrão,Sensib. média,Especif. média
DMC ['min-max'],0.756271,0.762712,0.627119,0.847458,0.044119,0.730282,0.832183


# 6. Classificador quadrático gaussiano (CQG)

Durante a implementação do CQG ocorreu um problema: o posto das matrizes de covariância das classes é menor do que a quantidade de atributos dos dados, o que foi verificado com o código a seguir.

In [17]:
from numpy.linalg import matrix_rank

# Cálculo das médias
m_park = np.matmul(features.values.T, (targets.values==1)) / sum(targets.values)
m_saud = np.matmul(features.values.T, (targets.values==0)) / (len(targets.values) - sum(targets.values))

# Cálculo das matrizes de covariância
C_park = np.zeros((features.shape[1],features.shape[1]))
C_saud = np.zeros((features.shape[1],features.shape[1]))

for j in range(len(targets.values)):
    if targets.values[j]: # indivíduo tem parkinson
        C_park += np.matmul(np.expand_dims((features.values[j,:]-m_park), axis=1),
                            np.expand_dims((features.values[j,:]-m_park), axis=0))
    else: # indivíduo saudável
        C_saud += np.matmul(np.expand_dims((features.values[j,:]-m_saud), axis=1),
                            np.expand_dims((features.values[j,:]-m_saud), axis=0))

# dividindo pelo número de elementos de cada classe
C_park/=sum(targets.values)
C_saud/=(len(targets.values) - sum(targets.values))

print("Número de atributos = {}".format(features.shape[1]))
print("Posto(C_parkinson) = {}\nPosto(C_saudavel) = {}".format(matrix_rank(C_park), matrix_rank(C_saud)))

Número de atributos = 22
Posto(C_parkinson) = 19
Posto(C_saudavel) = 19


Portanto foram utilizadas as seguintes metodologias para tornar as matrizes invertíveis:
- Variante 1: extrair apenas a diagonal principal das matrizes;
- Variante 2: matriz de covariância agregada (*pooled*);
- Variante 3: regularização de Friedman, nesse caso foi realizada um busca em grade para determinar o hiper-parâmetro $\lambda$.

Outro detalhe importante é que não foi necessário fazer a normalização dos dados pois ela é feita automaticamente pela matriz de covariância.

In [18]:
%%time
from numpy.linalg import inv
from numpy import dot, matmul

rodadas = 100

lambdas = np.linspace(0,1,num=10) # possíveis valores de lambda
nCases = 2+len(lambdas) # número de casos estudados

# para as 3 variantes
acc = np.zeros((nCases,rodadas))
especificidade = [0]*nCases
sensibilidade  = [0]*nCases
CQG_data = np.zeros((nCases, len(cabecalho)))
for i in range(rodadas):
    # divisão treino/teste 
    X_train, X_test, y_train, y_test = train_test_split(features.values, 
                                                        targets.values,
                                                        test_size=0.3)
    # Cálculo das médias
    m_park = np.matmul(X_train.T, (y_train==1)) / sum(y_train)
    m_saud = np.matmul(X_train.T, (y_train==0)) / (len(y_train) - sum(y_train))
    
    # Cálculo das matrizes de covariância
    C_park = np.zeros((features.shape[1],features.shape[1]))
    C_saud = np.zeros((features.shape[1],features.shape[1]))
        
    for j in range(len(y_train)):
        if y_train[j]: # indivíduo com doença de parkinson
            C_park += np.matmul(np.expand_dims((X_train[j,:]-m_park), axis=1),
                                np.expand_dims((X_train[j,:]-m_park), axis=0))
        else: # indivíduo saudável
            C_saud += np.matmul(np.expand_dims((X_train[j,:]-m_saud), axis=1),
                                np.expand_dims((X_train[j,:]-m_saud), axis=0))
    # dividindo pelo número de elementos de cada classe
    C_park/=sum(y_train)
    C_saud/=(len(y_train) - sum(y_train))
    
        
    # Predição do conjunto de teste
    y_pred = np.zeros((nCases,len(y_test)))
    
    # Variante 1:
    C_park_v1_inv = inv(np.diag(np.diag(C_park)))
    C_saud_v1_inv = inv(np.diag(np.diag(C_saud)))
        
    y_pred[0] = [
        1 if dot(matmul(
            (X_test[j]-m_park),C_park_v1_inv),(X_test[j]-m_park)) < dot(matmul(
            (X_test[j]-m_saud),C_saud_v1_inv),(X_test[j]-m_saud)) else 
        0 for j in range(len(y_test))]
        
            
    # Variante 2:
    C_pool = (sum(y_train)*C_park + (len(y_train)-sum(y_train))*C_saud )/len(y_train)
    C_pool_inv = inv(C_pool)
    
    y_pred[1] = [
        1 if dot(matmul(
            (X_test[j]-m_park),C_pool_inv),(X_test[j]-m_park)) < dot(matmul(
            (X_test[j]-m_saud),C_pool_inv),(X_test[j]-m_saud)) else 
        0 for j in range(len(y_test))]
    
    # Variante 3:
    for j in range(len(lambdas)):
        # calcular matriz regularizada
        C_park_v3_inv = inv(((1-lambdas[j])*sum(y_train)*C_park                + lambdas[j]*len(y_train)*C_pool) / 
                            ((1-lambdas[j])*sum(y_train)                       + lambdas[j]*len(y_train)) 
                           )
        C_saud_v3_inv = inv(((1-lambdas[j])*(len(y_train)-sum(y_train))*C_saud + lambdas[j]*len(y_train)*C_pool) / 
                            ((1-lambdas[j])*(len(y_train)-sum(y_train))        + lambdas[j]*len(y_train)) 
                           )
        
        # predição no conjunto de teste
        y_pred[j+2] = [
            1 if dot(matmul(
                (X_test[j]-m_park),C_park_v3_inv),(X_test[j]-m_park)) < dot(matmul(
                (X_test[j]-m_saud),C_saud_v3_inv),(X_test[j]-m_saud)) else 
            0 for j in range(len(y_test))]
    
    # Avaliação da performance
    for j in range(nCases):
        cm = confusion_matrix(y_test, y_pred[j])
        total=sum(sum(cm))

        acc[j][i] = (cm[0,0]+cm[1,1])/total
        especificidade[j] += cm[0,0]/(cm[0,0]+cm[0,1])
        sensibilidade[j]  += cm[1,1]/(cm[1,1]+cm[1,0])
        
# consolidando estatísticas
for j in range(nCases): 
    especificidade[j]/=rodadas # Valores médios
    sensibilidade[j] /=rodadas

    CQG_data[j] = np.matrix([np.mean(acc[j]), np.median(acc[j]), min(acc[j]), max(acc[j]),
                          np.std(acc[j]), sensibilidade[j], especificidade[j]])    
    
index = ["CQG variante 1 (diagonal)", "CQG variante 2 (pooled)"]
index.extend(["CQG variante 3 [$\lambda$ = {0:.3f}]".format(lambda_) for lambda_ in lambdas])
CQG_df = pd.DataFrame(CQG_data, columns=cabecalho, index=[index])

CPU times: user 8.78 s, sys: 15.7 s, total: 24.4 s
Wall time: 3.11 s


In [19]:
CQG_df

,Média,Mediana,Mínimo,Máximo,Desv. Padrão,Sensib. média,Especif. média
CQG variante 1 (diagonal),0.789831,0.796610,0.677966,0.915254,0.041861,0.826206,0.684266
CQG variante 2 (pooled),0.828475,0.830508,0.711864,0.915254,0.039480,0.860750,0.733134
CQG variante 3 [$\lambda$ = 0.000],0.818644,0.830508,0.694915,0.915254,0.055364,1.000000,0.285394
CQG variante 3 [$\lambda$ = 0.111],0.869831,0.864407,0.762712,0.983051,0.043535,0.920011,0.718591
CQG variante 3 [$\lambda$ = 0.222],0.842373,0.847458,0.745763,0.932203,0.042305,0.875913,0.740870
CQG variante 3 [$\lambda$ = 0.333],0.838136,0.830508,0.728814,0.949153,0.041056,0.867451,0.749622
CQG variante 3 [$\lambda$ = 0.444],0.835763,0.830508,0.745763,0.949153,0.039218,0.862688,0.755260
CQG variante 3 [$\lambda$ = 0.556],0.832881,0.830508,0.745763,0.932203,0.040325,0.858959,0.755736
CQG variante 3 [$\lambda$ = 0.667],0.831186,0.830508,0.745763,0.932203,0.038420,0.857956,0.751410
CQG variante 3 [$\lambda$ = 0.778],0.833898,0.830508,0.728814,0.932203,0.039749,0.862926,0.747375


O melhor desempenho foi atingido através da regularização de Friedman com $\lambda=0,111$:

In [20]:
CQG_df.sort_values('Média', ascending=False).iloc[0,:].to_frame().transpose()

,Média,Mediana,Mínimo,Máximo,Desv. Padrão,Sensib. média,Especif. média
CQG variante 3 [$\lambda$ = 0.111],0.869831,0.864407,0.762712,0.983051,0.043535,0.920011,0.718591


# 7. Redes Neurais Artificiais

## 7.1 Perceptron simples (PS)

O classificador PS foi implementado utilizando a biblioteca *numpy*. A função degrau foi escolhida como função de ativação e a regra de aprendizado utilizada para a atualização dos pesos foi:

$$\vec{w}(t+1) = \vec{w}(t) + \eta e(t) \vec{x}(t)$$

Abaixo segue o código que define a classe *PerceptronSimples*:

In [21]:
# Funções de ativação
from numpy import exp

def sinal(x):
    return +1 if x >= 0 else 0

def sigmoid(x):
    return 1/(1+exp(-x))

def tanHiper(x):
    return (1-exp(-x))/(1+exp(-x))

# Versão vetorial das funções:
vSinal    = np.vectorize(sinal)
vSigmoid  = np.vectorize(sigmoid)
vTanHiper = np.vectorize(tanHiper)

In [22]:
import plotly.offline as plt
import plotly.graph_objs as go

plt.init_notebook_mode(connected=True) # habilitando plot dentro do jupyter notebook

# Classe dos perceptrons simples
class PerceptronSimples:
    'Classe criada para descrever as redes de n perceptrons simples em paralelo.'
    
    def __init__(self, nEntradas, nSaidas):
        self.nEntradas = nEntradas
        self.nSaidas   = nSaidas
        self.param = None
        self.trainHist = {
            "train": {"tx": [], "cm": []}, 
            "test":  {"tx": [], "cm": []} 
        } 
        
        
    def initParam(self): # inicialização aleatória dos pesos por distribuição uniforme
        self.param = np.random.rand(self.nEntradas+1, self.nSaidas) # +1 por causa do bias
        # limpeza do histórico de treino
        self.trainHist = {
            "train": {"tx": [], "cm": []}, 
            "test":  {"tx": [], "cm": []} 
        } 
        
    def prever(self, X): # Y = [-1_{nAmostrasx1} X]*w
                         # As amostras na matriz X estão nas linhas, ou deitadas
                         # As variáveis então são as colunas
        XplusBias = np.concatenate((-np.ones((len(X),1)), X), axis=1)
        U = np.matmul(XplusBias, self.param) # ativação
        Y = vSinal(U) # saída do perceptron
        return Y
    
    def treinar(self, X_tr, Y_tr, X_ts, Y_ts, eta, nEpocas):
        for _ in range(nEpocas):
            # Embaralhar vetores de treinamento
            nAmostras = X_tr.shape[0]
            novosIndices = np.arange(nAmostras)
            np.random.shuffle(novosIndices)
            X = X_tr[novosIndices]
            Y = Y_tr[novosIndices]

            # Treino step-by-step
            for i in range(len(X)):
                amostra = X[i,:].reshape(1,len(X[i,:])) # colocando amostra como vetor linha
                Y_pred = self.prever(amostra) 
                E = Y[i] - Y_pred # erro da perceptron na amostra em questão
                self.param += eta*np.concatenate((-np.ones((1,len(amostra))), amostra), axis=1).transpose()*E
                
            self.updateHist(X_tr, Y_tr, X_ts, Y_ts)
    
    def updateHist(self, X_tr, Y_tr, X_ts, Y_ts):
        # Atualizando o histórico de treino da rede
        # conjunto de treino:
        tr_tx, tr_cm = self.taxaAcerto(Y_tr, self.prever(X_tr))
        self.trainHist["train"]["tx"].append(tr_tx)
        self.trainHist["train"]["cm"].append(tr_cm)
        # conjunto de teste:
        ts_tx, ts_cm = self.taxaAcerto(Y_ts, self.prever(X_ts))
        self.trainHist["test"]["tx"].append(ts_tx)
        self.trainHist["test"]["cm"].append(ts_cm)
            
    def plotTrainHist(self):
        epocas = [i for i in range(1,1+len(self.trainHist["train"]["tx"]))]

        # Gráfico da taxa de acerto ao longo das épocas
        traceTrTx = go.Scatter(
            x = epocas, 
            y = self.trainHist["train"]["tx"], 
            mode='lines+markers',
            name='conjunto de treino'
        )
        traceTsTx = go.Scatter(
            x = epocas, 
            y = self.trainHist["test"]["tx"], 
            mode='lines+markers',
            name='conjunto de teste'
        )
        data = [traceTrTx, traceTsTx]

        layoutTx = go.Layout(
            title = "Taxas de acerto ao longo do treinamento",
            legend=dict(orientation="h", y=-.05),
            xaxis=dict(title="Época"),
            yaxis=dict(title="Taxa de acerto [0,1]")
        )

        fig = go.Figure(data=data, layout=layoutTx)
        plt.iplot(fig)

        # Gráfico da matriz de confusão ao longo das épocas
        for nome in ["train","test"]:
            vn, fp, fn, vp = [],[],[],[]
            for cm in self.trainHist[nome]["cm"]:
                vn.append(cm[0,0])
                fp.append(cm[0,1])
                fn.append(cm[1,0])
                vp.append(cm[1,1])

            traceVn = go.Scatter(x = epocas, y = vn, mode='lines+markers', name='verdadeiros negativos')
            traceFp = go.Scatter(x = epocas, y = fp, mode='lines+markers', name='falsos positivos')
            traceFn = go.Scatter(x = epocas, y = fn, mode='lines+markers', name='falsos negativos')
            traceVp = go.Scatter(x = epocas, y = vp, mode='lines+markers', name='verdadeiros positivos')

            data = [traceVp, traceVn, traceFp, traceFn]

            layoutTx = go.Layout(
                title = "Matriz de confusão ao longo do treinamento ["+nome+"]",
                legend=dict(orientation="h", y=-.05),
                xaxis=dict(title="Época"),
                yaxis=dict(title="Taxa [0,1]")
            )

            fig = go.Figure(data=data, layout=layoutTx)
            plt.iplot(fig)
            

    def taxaAcerto(self, Y_real, Y_pred):
        Y_pred = np.squeeze(np.asarray(Y_pred))
        Y_real = np.squeeze(np.asarray(Y_real))
        diff = Y_pred - Y_real
        tx_acerto = 1 - np.absolute(diff).sum()/diff.size/2 # taxa de acerto \in [0,1]
        cm = confusion_matrix(Y_real, Y_pred)/len(Y_real) # divide-se pelo número de amostras para \in [0,1]
        return (tx_acerto, cm)

Como exemplo da classe *PerceptronSimples* em funcionamento o código abaixo foi elaborado para ilustrar o processo de aprendizado do classificador para uma rodada de separação treino/teste.

In [23]:
ps = PerceptronSimples(22,1)
ps.initParam()

# divisão treino/teste 
X_train, X_test, y_train, y_test = train_test_split(features.values, 
                                                    targets.values,
                                                    test_size=0.3)
# escalonar os dados
X_tr_norm, X_ts_norm = scale_feat(X_train, X_test, scaleType='min-max')

ps.treinar(X_tr_norm, y_train, X_ts_norm, y_test, eta=.001/2, nEpocas=150)

ps.plotTrainHist()

Posteriormente o classificador foi avaliado em 100 rodadas independentes de treino/teste. Como hiper-parâmetro foi analisado o efeito to tipo de normalização dos atributos.

In [24]:
%%time

# matriz que guardará os resultados numéricos
ps_data = np.zeros((len(scales), len(cabecalho))) 

for j in range(len(scales)):
    acc = [0]*100
    especificidade = 0
    sensibilidade  = 0
    for i in range(100):
        # divisão treino/teste 
        X_train, X_test, y_train, y_test = train_test_split(features.values, 
                                                            targets.values,
                                                            test_size=0.3)
        # escalonar os dados
        X_tr_norm, X_ts_norm = scale_feat(X_train, X_test, scaleType=scales[j])
        
        # construindo o classificador
        ps = PerceptronSimples(22,1)
        ps.initParam()
        ps.treinar(X_tr_norm, y_train, X_ts_norm, y_test, eta=.001/2, nEpocas=150)

        # calculando as métricas de avaliação
        cm = confusion_matrix(y_test, ps.prever(X_ts_norm))
        total=sum(sum(cm))

        acc[i] = (cm[0,0]+cm[1,1])/total
        especificidade += cm[0,0]/(cm[0,0]+cm[0,1])
        sensibilidade  += cm[1,1]/(cm[1,1]+cm[1,0])

    especificidade/=100 # Valores médios
    sensibilidade /=100
    
    ps_data[j,:] = np.matrix([np.mean(acc), np.median(acc), min(acc), max(acc), 
                         np.std(acc), sensibilidade, especificidade])

index = ['PS [\'{}\']'.format(scale) for scale in scales]
ps_df = pd.DataFrame(ps_data, columns=cabecalho, index=index)

CPU times: user 3min 50s, sys: 23 ms, total: 3min 50s
Wall time: 3min 50s


In [25]:
ps_df

,Média,Mediana,Mínimo,Máximo,Desv. Padrão,Sensib. média,Especif. média
PS ['min-max'],0.787797,0.788136,0.661017,0.898305,0.043575,0.863032,0.572168
PS ['std'],0.783051,0.779661,0.661017,0.898305,0.046293,0.863377,0.544861


Como pode ser visto na tabela o melhor desempenho ocorreu quanto a normalização foi do tipo *'min-max'*.

In [26]:
ps_df.sort_values('Média', ascending=False).iloc[0,:].to_frame().transpose()

,Média,Mediana,Mínimo,Máximo,Desv. Padrão,Sensib. média,Especif. média
PS ['min-max'],0.787797,0.788136,0.661017,0.898305,0.043575,0.863032,0.572168


## 7.2 Perceptron multicamadas (MLP)

Similar ao caso do PS o classificador MLP foi implementado na classe *PerceptronMulticamada*. Como função de ativação foi utilizada a função tangente hiperbólica e como regra de aprendizado o gradiente descendente com *backpropagation*.

In [27]:
# Classe dos Perceptrons Multicamadas
class PerceptronMulticamada(PerceptronSimples):
    def __init__(self, nEntradas, nOcultas, nSaidas):
        PerceptronSimples.__init__(self, nEntradas, nSaidas)
        self.nOcultas = nOcultas
                
    def initParam(self): # inicialização aleatória dos pesos por distribuição uniforme
        self.param = [np.random.rand(self.nEntradas+1, self.nOcultas)-.5,
                      np.random.rand(self.nOcultas+1, self.nSaidas)-.5]   # +1 por causa do bias
        # limpeza do histórico de treino
        self.trainHist = {"train": {"tx": [], "cm": []}, "test":  {"tx": [], "cm": []}} 
        
    def prever(self, X): # Y = [-1_{nAmostrasx1} X]*w
                         # As amostras na matriz X estão nas linhas, ou deitadas
                         # As variáveis então são as colunas
        XplusBias = np.concatenate((-np.ones((len(X),1)), X), axis=1)
        U1 = np.matmul(XplusBias, self.param[0]) # ativação da primeira camada
        Y1 = vTanHiper(U1)                       # saída da camada oculta
        
        XplusBias = np.concatenate((-np.ones((len(Y1),1)), Y1), axis=1)
        U2 = np.matmul(XplusBias, self.param[1]) # ativação da camada de saída
        Y2 = vTanHiper(U2)                       # saída da MLP
        return Y2, Y1
    
    def treinar(self, X_tr, Y_tr, X_ts, Y_ts, eta, nEpocas):
        for e in range(nEpocas):
            # Embaralhar vetores de treinamento
            novosIndices = np.arange(X_tr.shape[0])
            np.random.shuffle(novosIndices)
            X, Y = X_tr[novosIndices], Y_tr[novosIndices]

            # Treino step-by-step
            for i in range(len(X)):
                amostra = X[i,:].reshape(1,len(X[i,:])) # colocando amostra como vetor linha
                Y_pred, H_pred = self.prever(amostra)
                
                # Cálculo do erro e gradientes
                E = Y[i] - Y_pred # erro da MLP na amostra em questão
                #print(Y_pred.shape, E.shape)
                
                Dk = 0.5*(1 - np.power(Y_pred, 2)) # derivada da tangente hiperbólica (camada de saida)
                DDk = E*Dk               # gradiente local (camada de saida)
                
                Di = 0.5*(1 - np.power(H_pred,2).transpose())     # derivada da sigmoide logistica (camada oculta)
                DDi = np.multiply(Di, np.matmul(self.param[1][1:, :], DDk)) # gradiente local (camada oculta)
                
                # Atualização dos pesos
                self.param[1] += eta*DDk*np.concatenate((-np.ones((1,len(H_pred))), H_pred), axis=1).transpose()
                self.param[0] += eta*np.concatenate((-np.ones((1,len(amostra))), amostra), axis=1).transpose()*DDi.transpose()
                
            # Atualizando histórico
            self.updateHist(X_tr, Y_tr, X_ts, Y_ts)

    def taxaAcerto(self, Y_real, Y_pred):
        Y_pred = vSinal(Y_pred[0])
        diff = Y_pred - Y_real
        tx_acerto = 1 - np.absolute(diff).sum()/diff.size/2 # taxa de acerto \in [0,1]
        cm = confusion_matrix(Y_real, Y_pred)/len(Y_real) # divide-se pelo número de amostras para \in [0,1]
        return (tx_acerto, cm)

Para exemplificar a classe *PerceptronMulticamada* em funcionamento o código abaixo foi executado para gerar os gráficos das figuras a seguir.

In [28]:
%%time
# divisão treino/teste 
X_train, X_test, y_train, y_test = train_test_split(features.values, 
                                                    targets.values,
                                                    test_size=0.3)
# escalonar os dados
X_tr_norm, X_ts_norm = scale_feat(X_train, X_test, scaleType='min-max')

# Construção e treino do modelo
mlp = PerceptronMulticamada(22, 100, 1)
mlp.initParam()
mlp.treinar(X_tr_norm, y_train, X_ts_norm, y_test, eta=1e-4, nEpocas=150)
mlp.plotTrainHist()

CPU times: user 1min 9s, sys: 1min 43s, total: 2min 53s
Wall time: 29.5 s


Posteriormente o classificador MLP foi avaliado em 100 rodadas independentes de treino/teste. Como hiper-parâmetro só foi analisado a influência do tipo de normalização dos atributos. O número de neurônios na camada oculta, 100, fora escolhido por tentantiva e erro, uma busca em grade por esse hiper-parâmetro levaria muito tempo visto que o código abaixo necessitou de quase 1h e 30 minutos de processamento.

In [29]:
%%time

# matriz que guardará os resultados numéricos
mlp_data = np.zeros((len(scales), len(cabecalho))) 

for j in range(len(scales)):
    acc = [0]*100
    especificidade = 0
    sensibilidade  = 0
    for i in range(100):
        # divisão treino/teste 
        X_train, X_test, y_train, y_test = train_test_split(features.values, 
                                                            targets.values,
                                                            test_size=0.3)
        # escalonar os dados
        X_tr_norm, X_ts_norm = scale_feat(X_train, X_test, scaleType=scales[j])
        
        # construindo o classificador
        mlp = PerceptronMulticamada(22, 100, 1)
        mlp.initParam()
        mlp.treinar(X_tr_norm, y_train, X_ts_norm, y_test, eta=1e-4, nEpocas=150)

        # calculando as métricas de avaliação
        cm = confusion_matrix(y_test, mlp.prever(X_ts_norm)[0]>0)
        total=sum(sum(cm))

        acc[i] = (cm[0,0]+cm[1,1])/total
        especificidade += cm[0,0]/(cm[0,0]+cm[0,1])
        sensibilidade  += cm[1,1]/(cm[1,1]+cm[1,0])

    especificidade/=100 # Valores médios
    sensibilidade /=100
    
    mlp_data[j,:] = np.matrix([np.mean(acc), np.median(acc), min(acc), max(acc), 
                         np.std(acc), sensibilidade, especificidade])

index = ['MLP [\'{}\']'.format(scale) for scale in scales]
mlp_df = pd.DataFrame(mlp_data, columns=cabecalho, index=index)

CPU times: user 3h 50min 45s, sys: 5h 58min 3s, total: 9h 48min 49s
Wall time: 1h 32min 40s


In [30]:
mlp_df

,Média,Mediana,Mínimo,Máximo,Desv. Padrão,Sensib. média,Especif. média
MLP ['min-max'],0.745763,0.745763,0.627119,0.881356,0.049935,1.000000,0.000000
MLP ['std'],0.780000,0.779661,0.627119,0.881356,0.048118,0.987775,0.182239


Como podemos ver o MLP tem melhor desempenho quando feito uma normalização do tipo *'std'*.

In [31]:
mlp_df.sort_values('Média', ascending=False).iloc[0,:].to_frame().transpose()

,Média,Mediana,Mínimo,Máximo,Desv. Padrão,Sensib. média,Especif. média
MLP ['std'],0.78,0.779661,0.627119,0.881356,0.048118,0.987775,0.182239


# 8. Redução de dimensionalidade

Neste tópico foram utilizados dois algoritmos de redução de dimensionalidade: *Principal Component Analysis* (PCA) e *Linear Discriminant Analysis* (LDA). Para a escolha da dimensão em que os dados serão projetados foi aplicado o PCA no conjunto de dados completo, com a variância conservada em 98%. O código abaixo foi utilizado para estimar o número de componentes:

In [32]:
from sklearn.decomposition import PCA

p = .98
pca = PCA(n_components=p)
pca.fit(features.values)
print("Porcentagem mínima de variância a ser conservada: {}%".format(p*100))
print("Número de dimensões: {}".format(len(pca.components_)))

Porcentagem mínima de variância a ser conservada: 98.0%
Número de dimensões: 3


Consequentemente, tanto para o PCA quanto para o LDA, os dados foram projetados em espaços tridimensionais e todos os classificadores foram reaplicados nos dados transformados. Para evitar que informações do conjunto de teste vazassem para os classificadores, em cada rodada independente de treino/teste, os dados foram primeiro separados entre treino e teste e depois o PCA e LDA foram construídos a partir dos dados de treinamento. Os dados de treino foram transformados, os classificadores construídos e então, antes de fazer as previsões no conjunto de teste, os dados de teste foram transformados utilizando o PCA/LDA criado anteriormente.

Uma classe foi implementada para facilitar o uso do LDA de Fisher:

In [33]:
class LDA():
    'Implenta o LDA de Fisher para um problema de 2 classes'
    
    def __init__(self, n_components):
        self.n_components = n_components
        self.W = None
        
    def fit(self, X, y):
        import warnings
        warnings.filterwarnings('ignore')
        
        n_features = X.shape[1]
        # centróides de cada classe
        m0 = np.mean(X[y==0], axis=0)
        m1 = np.mean(X[y==1], axis=0)
        
        # Cálculo das matrizes de covariância
        C_0 = np.zeros((n_features,n_features))
        C_1 = np.zeros((n_features,n_features))

        for j in range(len(y)):
            if y[j]==0:
                C_0 += np.matmul(np.expand_dims((X[j,:]-m0), axis=1),
                                 np.expand_dims((X[j,:]-m0), axis=0))
            else: # y[j]==1
                C_1 += np.matmul(np.expand_dims((X[j,:]-m1), axis=1),
                                 np.expand_dims((X[j,:]-m1), axis=0))
        # dividindo pelo número de elementos de cada classe
        n0 = (len(y)-sum(y))
        n1 = sum(y)
        C_0/=n0
        C_1/=n1
                
        # Matriz de dispersão intraclasse S_W
        S_W = n0*C_0 + n1*C_1
        
        # Matriz de disperão interclasse S_B
        m_all = np.mean(X, axis=0).reshape(n_features,1) # centróide dos dados
        m0 = m0.reshape(n_features,1)
        m1 = m1.reshape(n_features,1)
        S_B = (m0-m_all).dot((m0-m_all).T) + (m1-m_all).dot((m1-m_all).T)
        
        # autovalores e autovetores de S_W^-1*S_B
        eig_vals, eig_vecs = np.linalg.eig(np.linalg.inv(S_W).dot(S_B))
        # ordenando pares
        eig_pairs = [(np.abs(eig_vals[i]), eig_vecs[:,i]) for i in range(len(eig_vals))]
        eig_pairs = sorted(eig_pairs, key=lambda k: k[0], reverse=True)
        
        # Matriz de transformação dos dados
        self.W = np.zeros((n_features, self.n_components))
        for i in range(self.n_components):
            self.W[:,i] = eig_pairs[i][1]#.reshape(n_features,1)
        #= np.hstack((eig_pairs[0][1].reshape(4,1), eig_pairs[1][1].reshape(4,1))) 
               
        
    def transform(self, X):
        return X.dot(self.W)

## 8.1 PCA/LDA + k-NN

Novamente fora realizada uma busca dos hiper-parâmetros $k$ e o tipo de escalonamento dos dados.

In [34]:
%%time
from sklearn.decomposition import PCA

rodadas = 100

scales = ['min-max', 'std']   # tipos de escalonamentos possíveis
ks = [i for i in range(1,11)] # valores possíveis de k para k-NN
pca_nn_data = np.zeros((len(ks)*len(scales), len(cabecalho))) # matriz que guardará resultados numéricos
lda_nn_data = np.zeros((len(ks)*len(scales), len(cabecalho))) # matriz que guardará resultados numéricos

for scale in scales:
    for k in ks:
        pca_especificidade = 0
        lda_especificidade = 0
        
        pca_sensibilidade  = 0
        lda_sensibilidade  = 0
        
        pca_acc = [0]*rodadas
        lda_acc = [0]*rodadas
        for i in range(rodadas):
            # divisão treino/teste 
            X_train, X_test, y_train, y_test = train_test_split(features.values, 
                                                                targets.values,
                                                                test_size=0.3)
            # PCA
            pca = PCA(n_components = 3)
            pca.fit(X_train)
            
            pca_X_train = pca.transform(X_train)
            pca_X_test  = pca.transform(X_test)
            
            
            # LDA
            lda = LDA(n_components = 3)
            lda.fit(X_train, y_train)
            
            lda_X_train = lda.transform(X_train)
            lda_X_test  = lda.transform(X_test)
            
            # escalonamento dos dados
            pca_X_tr_norm, pca_X_ts_norm = scale_feat(pca_X_train, pca_X_test, scaleType=scale)
            lda_X_tr_norm, lda_X_ts_norm = scale_feat(lda_X_train, lda_X_test, scaleType=scale)

            # construindo os classificadores
            pca_k_nn = KNeighborsClassifier(n_neighbors=k, n_jobs=-1)
            pca_k_nn.fit(pca_X_tr_norm, y_train)
            
            lda_k_nn = KNeighborsClassifier(n_neighbors=k, n_jobs=-1)
            lda_k_nn.fit(lda_X_tr_norm, y_train)

            # calculando as métricas de avaliação
            pca_cm = confusion_matrix(y_test, pca_k_nn.predict(pca_X_ts_norm))
            total=sum(sum(pca_cm))

            pca_acc[i] = (pca_cm[0,0]+pca_cm[1,1])/total
            pca_especificidade += pca_cm[0,0]/(pca_cm[0,0]+pca_cm[0,1])
            pca_sensibilidade  += pca_cm[1,1]/(pca_cm[1,1]+pca_cm[1,0])
            
            lda_cm = confusion_matrix(y_test, lda_k_nn.predict(lda_X_ts_norm))
            total=sum(sum(lda_cm))

            lda_acc[i] = (lda_cm[0,0]+lda_cm[1,1])/total
            lda_especificidade += lda_cm[0,0]/(lda_cm[0,0]+lda_cm[0,1])
            lda_sensibilidade  += lda_cm[1,1]/(lda_cm[1,1]+lda_cm[1,0])

                        
        pca_especificidade/=rodadas # Valores médios
        pca_sensibilidade /=rodadas
        
        lda_especificidade/=rodadas # Valores médios
        lda_sensibilidade /=rodadas

        index = k-1 if scale=='min-max' else k+9 # indíce da linha dos dados
        pca_nn_data[index,:] = np.matrix([np.mean(pca_acc), np.median(pca_acc), min(pca_acc), max(pca_acc), 
                                          np.std(pca_acc), pca_sensibilidade, pca_especificidade])
        
        lda_nn_data[index,:] = np.matrix([np.mean(lda_acc), np.median(lda_acc), min(lda_acc), max(lda_acc), 
                                          np.std(lda_acc), lda_sensibilidade, lda_especificidade])
        

pca_idx_label      = ['PCA+{}-NN [\'min-max\']'.format(k) for k in ks]
pca_idx_label.extend(['PCA+{}-NN [\'std\']'.format(k) for k in ks])
df_knn_pca = pd.DataFrame(pca_nn_data, columns=cabecalho, index=[pca_idx_label])

lda_idx_label      = ['LDA+{}-NN [\'min-max\']'.format(k) for k in ks]
lda_idx_label.extend(['LDA+{}-NN [\'std\']'.format(k) for k in ks])
df_knn_lda = pd.DataFrame(lda_nn_data, columns=cabecalho, index=[lda_idx_label])

CPU times: user 7min 41s, sys: 18min 14s, total: 25min 56s
Wall time: 7min 48s


In [35]:
display(df_knn_pca)
display(df_knn_lda)

,Média,Mediana,Mínimo,Máximo,Desv. Padrão,Sensib. média,Especif. média
PCA+1-NN ['min-max'],0.822034,0.813559,0.728814,0.932203,0.045321,0.881408,0.644347
PCA+2-NN ['min-max'],0.773729,0.771186,0.661017,0.898305,0.049553,0.792269,0.727454
PCA+3-NN ['min-max'],0.823729,0.830508,0.711864,0.932203,0.040607,0.925959,0.523732
PCA+4-NN ['min-max'],0.825085,0.830508,0.711864,0.915254,0.040172,0.895563,0.625149
PCA+5-NN ['min-max'],0.830169,0.830508,0.711864,0.949153,0.044132,0.947021,0.478715
PCA+6-NN ['min-max'],0.824746,0.830508,0.694915,0.915254,0.043293,0.918691,0.540975
PCA+7-NN ['min-max'],0.822373,0.822034,0.711864,0.915254,0.044810,0.946138,0.427493
PCA+8-NN ['min-max'],0.808475,0.813559,0.711864,0.932203,0.044295,0.925452,0.465548
PCA+9-NN ['min-max'],0.816441,0.813559,0.711864,0.881356,0.034240,0.958256,0.362951
PCA+10-NN ['min-max'],0.816102,0.813559,0.711864,0.932203,0.041543,0.945545,0.419958


,Média,Mediana,Mínimo,Máximo,Desv. Padrão,Sensib. média,Especif. média
LDA+1-NN ['min-max'],0.843390,0.847458,0.728814,0.932203,0.040898,0.891525,0.700874
LDA+2-NN ['min-max'],0.805424,0.813559,0.627119,0.949153,0.047122,0.822707,0.759978
LDA+3-NN ['min-max'],0.859831,0.864407,0.762712,0.932203,0.039955,0.922350,0.674417
LDA+4-NN ['min-max'],0.851017,0.864407,0.762712,0.949153,0.043431,0.895236,0.728742
LDA+5-NN ['min-max'],0.859831,0.864407,0.745763,0.932203,0.042465,0.937843,0.624209
LDA+6-NN ['min-max'],0.859661,0.864407,0.711864,0.932203,0.040400,0.916731,0.692592
LDA+7-NN ['min-max'],0.870678,0.881356,0.762712,0.949153,0.039142,0.939786,0.646923
LDA+8-NN ['min-max'],0.857797,0.864407,0.762712,0.932203,0.035786,0.929870,0.648249
LDA+9-NN ['min-max'],0.877966,0.881356,0.745763,0.949153,0.035553,0.956349,0.625656
LDA+10-NN ['min-max'],0.871525,0.881356,0.745763,0.966102,0.037685,0.942844,0.649274


De forma geral os resultados do k-NN quando aplicado PCA e LDA foram piores. Os melhores resultados foram:

In [36]:
display(df_knn_pca.sort_values('Média', ascending=False).iloc[0,:].to_frame().transpose())
display(df_knn_lda.sort_values('Média', ascending=False).iloc[0,:].to_frame().transpose())

,Média,Mediana,Mínimo,Máximo,Desv. Padrão,Sensib. média,Especif. média
PCA+5-NN ['min-max'],0.830169,0.830508,0.711864,0.949153,0.044132,0.947021,0.478715


,Média,Mediana,Mínimo,Máximo,Desv. Padrão,Sensib. média,Especif. média
LDA+9-NN ['std'],0.878305,0.881356,0.79661,0.932203,0.033676,0.957085,0.629691


## 8.2 PCA/LDA + Reg. Linear

Analogamente ao caso sem redução de dimensionalidade fora avaliado a influência do tipo de escalonamento dos dados.

In [37]:
%%time

rodadas = 100

# matriz que guardará os resultados numéricos
pca_reg_data = np.zeros((len(scales), len(cabecalho)))
lda_reg_data = np.zeros((len(scales), len(cabecalho)))

for j in range(len(scales)):
    pca_acc = [0]*rodadas
    lda_acc = [0]*rodadas
    
    pca_especificidade = 0
    lda_especificidade = 0
    
    pca_sensibilidade  = 0
    lda_sensibilidade  = 0
    
    for i in range(rodadas):
        # divisão treino/teste 
        X_train, X_test, y_train, y_test = train_test_split(features.values, 
                                                            targets.values,
                                                            test_size=0.3)
        # PCA
        pca = PCA(n_components = 3)
        pca.fit(X_train)

        pca_X_train = pca.transform(X_train)
        pca_X_test  = pca.transform(X_test)

        # LDA
        lda = LDA(n_components = 3)
        lda.fit(X_train, y_train)

        lda_X_train = lda.transform(X_train)
        lda_X_test  = lda.transform(X_test)

        # escalonamento dos dados
        pca_X_tr_norm, pca_X_ts_norm = scale_feat(pca_X_train, pca_X_test, scaleType=scales[j])
        lda_X_tr_norm, lda_X_ts_norm = scale_feat(lda_X_train, lda_X_test, scaleType=scales[j])

        
        # construindo os classificadores
        pca_reg = linear_model.LinearRegression(n_jobs=-1)
        pca_reg.fit(pca_X_tr_norm, y_train)
        
        lda_reg = linear_model.LinearRegression(n_jobs=-1)
        lda_reg.fit(lda_X_tr_norm, y_train)

        # calculando métricas
        pca_cm = confusion_matrix(y_test, pca_reg.predict(pca_X_ts_norm)>=.5)
        total=sum(sum(pca_cm))

        pca_acc[i] = (pca_cm[0,0]+pca_cm[1,1])/total
        pca_especificidade += pca_cm[0,0]/(pca_cm[0,0]+pca_cm[0,1])
        pca_sensibilidade  += pca_cm[1,1]/(pca_cm[1,1]+pca_cm[1,0])
        
        lda_cm = confusion_matrix(y_test, lda_reg.predict(lda_X_ts_norm)>=.5)
        total=sum(sum(lda_cm))

        lda_acc[i] = (lda_cm[0,0]+lda_cm[1,1])/total
        lda_especificidade += lda_cm[0,0]/(lda_cm[0,0]+lda_cm[0,1])
        lda_sensibilidade  += lda_cm[1,1]/(lda_cm[1,1]+lda_cm[1,0])

    pca_especificidade/=rodadas # Valores médios
    pca_sensibilidade /=rodadas
    
    lda_especificidade/=rodadas # Valores médios
    lda_sensibilidade /=rodadas
    
    pca_reg_data[j,:] = np.matrix([np.mean(pca_acc), np.median(pca_acc), min(pca_acc), max(pca_acc), 
                                   np.std(pca_acc), pca_sensibilidade, pca_especificidade])
    lda_reg_data[j,:] = np.matrix([np.mean(lda_acc), np.median(lda_acc), min(lda_acc), max(lda_acc), 
                                   np.std(lda_acc), lda_sensibilidade, lda_especificidade])

pca_index = ['PCA+Reg. Linear [\'{}\']'.format(scale) for scale in scales]
pca_reg_df = pd.DataFrame(pca_reg_data, columns=cabecalho, index=pca_index)
lda_index = ['LDA+Reg. Linear [\'{}\']'.format(scale) for scale in scales]
lda_reg_df = pd.DataFrame(lda_reg_data, columns=cabecalho, index=lda_index)

CPU times: user 4.99 s, sys: 8.28 s, total: 13.3 s
Wall time: 1.67 s


In [38]:
display(pca_reg_df)
display(lda_reg_df)

,Média,Mediana,Mínimo,Máximo,Desv. Padrão,Sensib. média,Especif. média
PCA+Reg. Linear ['min-max'],0.817966,0.813559,0.711864,0.915254,0.043187,0.965156,0.382790
PCA+Reg. Linear ['std'],0.810339,0.813559,0.694915,0.915254,0.042589,0.957207,0.371852


,Média,Mediana,Mínimo,Máximo,Desv. Padrão,Sensib. média,Especif. média
LDA+Reg. Linear ['min-max'],0.869322,0.881356,0.779661,0.966102,0.034008,0.948103,0.638659
LDA+Reg. Linear ['std'],0.871017,0.872881,0.779661,0.932203,0.037511,0.949311,0.639418


Enquanto a versão com PCA teve um desempenho pior que a original, a versão com LDA apresentou um desempenho equivalente ao dos dados não transformados. Segue abaixo os melhores desempenhos para "PCA+Reg. Linear" e "LDA+Reg. Linear":

In [39]:
display(pca_reg_df.sort_values('Média', ascending=False).iloc[0,:].to_frame().transpose())
display(lda_reg_df.sort_values('Média', ascending=False).iloc[0,:].to_frame().transpose())

,Média,Mediana,Mínimo,Máximo,Desv. Padrão,Sensib. média,Especif. média
PCA+Reg. Linear ['min-max'],0.817966,0.813559,0.711864,0.915254,0.043187,0.965156,0.38279


,Média,Mediana,Mínimo,Máximo,Desv. Padrão,Sensib. média,Especif. média
LDA+Reg. Linear ['std'],0.871017,0.872881,0.779661,0.932203,0.037511,0.949311,0.639418


## 8.3 PCA/LDA + Reg. Logística

A mesma busca em grade pelos hiper-parâmetros do modelo fora realizada.

In [40]:
%%time

rodadas = 100

# Hiper-parâmetros
scales = ['min-max', 'std']
penalties = ['l1', 'l2']
Cs = [10**(i) for i in range(-3,5)]

nCases = len(scales)*len(penalties)*len(Cs) # número de combinações dos hiperparâmetros
pca_reg_log_data = np.zeros((nCases,len(cabecalho)))
pca_index = [' ']*nCases # lista de índices a serem salvos
lda_reg_log_data = np.zeros((nCases,len(cabecalho)))
lda_index = [' ']*nCases # lista de índices a serem salvos

count = 0
for scale in scales:
    for penalty in penalties:
        for C in Cs:       
            pca_acc = [0]*rodadas
            lda_acc = [0]*rodadas
            pca_especificidade = 0
            lda_especificidade = 0
            pca_sensibilidade  = 0
            lda_sensibilidade  = 0
            for i in range(rodadas):
                # divisão treino/teste 
                X_train, X_test, y_train, y_test = train_test_split(features.values, 
                                                                    targets.values,
                                                                    test_size=0.3)
                # PCA
                pca = PCA(n_components = 3)
                pca.fit(X_train)
                pca_X_train = pca.transform(X_train)
                pca_X_test  = pca.transform(X_test)

                # LDA
                lda = LDA(n_components = 3)
                lda.fit(X_train, y_train)
                lda_X_train = lda.transform(X_train)
                lda_X_test  = lda.transform(X_test)

                # escalonamento dos dados
                pca_X_tr_norm, pca_X_ts_norm = scale_feat(pca_X_train, pca_X_test, scaleType=scale)
                lda_X_tr_norm, lda_X_ts_norm = scale_feat(lda_X_train, lda_X_test, scaleType=scale)

                # construindo classificador
                solver = 'lbfgs' if scale=='l2' else 'liblinear'
                pca_reg_log = LogisticRegression(penalty=penalty, C=C, solver=solver, max_iter=int(1e4))
                pca_reg_log.fit(pca_X_tr_norm, y_train)
                lda_reg_log = LogisticRegression(penalty=penalty, C=C, solver=solver, max_iter=int(1e4))
                lda_reg_log.fit(lda_X_tr_norm, y_train)
                
                # cálculo das métricas
                cm = confusion_matrix(y_test, pca_reg_log.predict(pca_X_ts_norm)>.5)
                total=sum(sum(cm))
                pca_acc[i] = (cm[0,0]+cm[1,1])/total
                pca_especificidade += cm[0,0]/(cm[0,0]+cm[0,1])
                pca_sensibilidade  += cm[1,1]/(cm[1,1]+cm[1,0])
                
                cm = confusion_matrix(y_test, lda_reg_log.predict(lda_X_ts_norm)>.5)
                total=sum(sum(cm))
                lda_acc[i] = (cm[0,0]+cm[1,1])/total
                lda_especificidade += cm[0,0]/(cm[0,0]+cm[0,1])
                lda_sensibilidade  += cm[1,1]/(cm[1,1]+cm[1,0])

            pca_especificidade/=rodadas # Valores médios
            pca_sensibilidade /=rodadas
            lda_especificidade/=rodadas
            lda_sensibilidade /=rodadas

            pca_reg_log_data[count,:] = np.matrix([np.mean(pca_acc), np.median(pca_acc), min(pca_acc), 
                                           max(pca_acc), np.std(pca_acc), pca_sensibilidade, 
                                           pca_especificidade])
            lda_reg_log_data[count,:] = np.matrix([np.mean(lda_acc), np.median(lda_acc), min(lda_acc), 
                                           max(lda_acc), np.std(lda_acc), lda_sensibilidade, 
                                           lda_especificidade])
        
            pca_index[count] = 'PCA+Reg. Log. [{} / {} / {}]'.format(scale, penalty, C)
            lda_index[count] = 'LDA+Reg. Log. [{} / {} / {}]'.format(scale, penalty, C)
            
            count+=1
                
pca_reg_log_df = pd.DataFrame(pca_reg_log_data, columns=cabecalho, index=[pca_index])
lda_reg_log_df = pd.DataFrame(lda_reg_log_data, columns=cabecalho, index=[lda_index])

CPU times: user 1min 24s, sys: 2min 24s, total: 3min 49s
Wall time: 28.7 s


In [41]:
display(pca_reg_log_df)
display(lda_reg_log_df)

,Média,Mediana,Mínimo,Máximo,Desv. Padrão,Sensib. média,Especif. média
PCA+Reg. Log. [min-max / l1 / 0.001],0.249153,0.245763,0.152542,0.372881,0.048208,0.000000,1.000000
PCA+Reg. Log. [min-max / l1 / 0.01],0.245424,0.254237,0.118644,0.338983,0.049326,0.000000,1.000000
PCA+Reg. Log. [min-max / l1 / 0.1],0.755254,0.745763,0.627119,0.881356,0.048849,1.000000,0.000000
PCA+Reg. Log. [min-max / l1 / 1],0.801695,0.813559,0.677966,0.932203,0.053248,0.991369,0.240483
PCA+Reg. Log. [min-max / l1 / 10],0.819661,0.830508,0.728814,0.932203,0.046263,0.972350,0.370661
PCA+Reg. Log. [min-max / l1 / 100],0.816780,0.813559,0.694915,0.915254,0.043059,0.966862,0.375161
PCA+Reg. Log. [min-max / l1 / 1000],0.818814,0.813559,0.694915,0.915254,0.044178,0.964630,0.373876
PCA+Reg. Log. [min-max / l1 / 10000],0.811525,0.813559,0.694915,0.915254,0.045623,0.963508,0.363357
PCA+Reg. Log. [min-max / l2 / 0.001],0.746780,0.745763,0.610169,0.881356,0.051175,1.000000,0.000000
PCA+Reg. Log. [min-max / l2 / 0.01],0.760169,0.762712,0.610169,0.864407,0.050642,1.000000,0.000000


,Média,Mediana,Mínimo,Máximo,Desv. Padrão,Sensib. média,Especif. média
LDA+Reg. Log. [min-max / l1 / 0.001],0.249153,0.245763,0.152542,0.372881,0.048208,0.000000,1.000000
LDA+Reg. Log. [min-max / l1 / 0.01],0.245424,0.254237,0.118644,0.338983,0.049326,0.000000,1.000000
LDA+Reg. Log. [min-max / l1 / 0.1],0.755763,0.745763,0.627119,0.881356,0.049246,1.000000,0.002338
LDA+Reg. Log. [min-max / l1 / 1],0.876102,0.881356,0.762712,0.949153,0.040015,0.966920,0.606916
LDA+Reg. Log. [min-max / l1 / 10],0.862712,0.864407,0.762712,0.949153,0.040854,0.935991,0.642836
LDA+Reg. Log. [min-max / l1 / 100],0.865593,0.864407,0.779661,0.949153,0.041257,0.936283,0.659517
LDA+Reg. Log. [min-max / l1 / 1000],0.868983,0.872881,0.728814,0.966102,0.039594,0.938624,0.656364
LDA+Reg. Log. [min-max / l1 / 10000],0.860678,0.864407,0.711864,0.966102,0.045263,0.932574,0.650362
LDA+Reg. Log. [min-max / l2 / 0.001],0.746780,0.745763,0.610169,0.881356,0.051175,1.000000,0.000000
LDA+Reg. Log. [min-max / l2 / 0.01],0.760169,0.762712,0.610169,0.864407,0.050642,1.000000,0.000000


Ordenando os resultados por ordem decrescente de média da taxa de acertos:

In [42]:
display(pca_reg_log_df.sort_values('Média', ascending=False).head())
display(lda_reg_log_df.sort_values('Média', ascending=False).head())

,Média,Mediana,Mínimo,Máximo,Desv. Padrão,Sensib. média,Especif. média
PCA+Reg. Log. [std / l2 / 10000],0.824237,0.830508,0.661017,0.915254,0.049103,0.959592,0.399696
PCA+Reg. Log. [std / l2 / 0.001],0.823559,0.830508,0.711864,0.932203,0.045295,0.971504,0.389342
PCA+Reg. Log. [min-max / l2 / 10000],0.822712,0.830508,0.677966,0.915254,0.044998,0.969267,0.382449
PCA+Reg. Log. [min-max / l2 / 1000],0.820847,0.830508,0.711864,0.932203,0.043823,0.963042,0.381824
PCA+Reg. Log. [min-max / l2 / 100],0.820339,0.813559,0.694915,0.932203,0.043278,0.965097,0.385246


,Média,Mediana,Mínimo,Máximo,Desv. Padrão,Sensib. média,Especif. média
LDA+Reg. Log. [min-max / l1 / 1],0.876102,0.881356,0.762712,0.949153,0.040015,0.966920,0.606916
LDA+Reg. Log. [min-max / l2 / 10],0.873390,0.881356,0.745763,0.966102,0.044551,0.951543,0.643449
LDA+Reg. Log. [std / l2 / 0.001],0.873051,0.881356,0.762712,0.932203,0.038239,0.956007,0.632693
LDA+Reg. Log. [min-max / l1 / 1000],0.868983,0.872881,0.728814,0.966102,0.039594,0.938624,0.656364
LDA+Reg. Log. [std / l2 / 0.01],0.867458,0.864407,0.762712,0.966102,0.044027,0.950986,0.619030


Enquanto os resultados com PCA foram piores que o caso original, os resultados com LDA foram melhores. Os melhores desempenhos de "PCA+Reg. Log." e "LDA+Reg. Log." seguem abaixo:

In [43]:
display(pca_reg_log_df.sort_values('Média', ascending=False).iloc[0,:].to_frame().transpose())
display(lda_reg_log_df.sort_values('Média', ascending=False).iloc[0,:].to_frame().transpose())

,Média,Mediana,Mínimo,Máximo,Desv. Padrão,Sensib. média,Especif. média
PCA+Reg. Log. [std / l2 / 10000],0.824237,0.830508,0.661017,0.915254,0.049103,0.959592,0.399696


,Média,Mediana,Mínimo,Máximo,Desv. Padrão,Sensib. média,Especif. média
LDA+Reg. Log. [min-max / l1 / 1],0.876102,0.881356,0.762712,0.949153,0.040015,0.96692,0.606916


## 8.4 PCA/LDA+DMC

Análogo ao caso dos dados originais fora analisada a influência do tipo de normalização nas métricas do classificador.

In [44]:
%%time

rodadas = 100

scales = ['min-max', 'std']
nCases = len(scales) # número de combinações dos hiperparâmetros
pca_DMC_data = np.zeros((nCases,len(cabecalho)))
lda_DMC_data = np.zeros((nCases,len(cabecalho)))
pca_index = [' ']*nCases # lista de índices a serem salvos
lda_index = [' ']*nCases

count = 0
for scale in scales:
    pca_acc = [0]*rodadas
    lda_acc = [0]*rodadas
    pca_especificidade = 0
    lda_especificidade = 0
    pca_sensibilidade  = 0
    lda_sensibilidade  = 0
    for i in range(rodadas):
        # divisão treino/teste 
        X_train, X_test, y_train, y_test = train_test_split(features.values, 
                                                            targets.values,
                                                            test_size=0.3)
        # PCA
        pca = PCA(n_components = 3)
        pca.fit(X_train)
        pca_X_train = pca.transform(X_train)
        pca_X_test  = pca.transform(X_test)

        # LDA
        lda = LDA(n_components = 3)
        lda.fit(X_train, y_train)
        lda_X_train = lda.transform(X_train)
        lda_X_test  = lda.transform(X_test)

        # escalonamento dos dados
        pca_X_tr_norm, pca_X_ts_norm = scale_feat(pca_X_train, pca_X_test, scaleType=scale)
        lda_X_tr_norm, lda_X_ts_norm = scale_feat(lda_X_train, lda_X_test, scaleType=scale)
        
        # Cálculo dos centróides
        pca_c_parkinson = np.matmul(pca_X_tr_norm.T, (y_train==1)) / sum(y_train)
        lda_c_parkinson = np.matmul(lda_X_tr_norm.T, (y_train==1)) / sum(y_train)
        pca_c_saudavel  = np.matmul(pca_X_tr_norm.T, (y_train==0)) / (len(y_train) - sum(y_train))
        lda_c_saudavel  = np.matmul(lda_X_tr_norm.T, (y_train==0)) / (len(y_train) - sum(y_train))
        
        # Predição do conjunto de teste
        pca_y_pred = [1 if norm(u-pca_c_parkinson) < norm(u-pca_c_saudavel) else 0 for u in pca_X_ts_norm]
        lda_y_pred = [1 if norm(u-lda_c_parkinson) < norm(u-lda_c_saudavel) else 0 for u in lda_X_ts_norm]
        
        # métricas de avaliação
        cm = confusion_matrix(y_test, pca_y_pred)
        total=sum(sum(cm))
        pca_acc[i] = (cm[0,0]+cm[1,1])/total
        pca_especificidade += cm[0,0]/(cm[0,0]+cm[0,1])
        pca_sensibilidade  += cm[1,1]/(cm[1,1]+cm[1,0])
        
        cm = confusion_matrix(y_test, lda_y_pred)
        total=sum(sum(cm))
        lda_acc[i] = (cm[0,0]+cm[1,1])/total
        lda_especificidade += cm[0,0]/(cm[0,0]+cm[0,1])
        lda_sensibilidade  += cm[1,1]/(cm[1,1]+cm[1,0])

    pca_especificidade/=rodadas # Valores médios
    lda_especificidade/=rodadas
    pca_sensibilidade /=rodadas
    lda_sensibilidade /=rodadas

    pca_DMC_data[count,:] = np.matrix([np.mean(pca_acc), np.median(pca_acc), min(pca_acc), 
                                       max(pca_acc), np.std(pca_acc), pca_sensibilidade, 
                                       pca_especificidade])
    lda_DMC_data[count,:] = np.matrix([np.mean(lda_acc), np.median(lda_acc), min(lda_acc), 
                                       max(lda_acc), np.std(lda_acc), lda_sensibilidade, 
                                       lda_especificidade])
    pca_index[count] = 'PCA+DMC [\'{}\']'.format(scale)
    lda_index[count] = 'LDA+DMC [\'{}\']'.format(scale)

    count+=1

pca_DMC_df = pd.DataFrame(pca_DMC_data, columns=cabecalho, index=[pca_index])
lda_DMC_df = pd.DataFrame(lda_DMC_data, columns=cabecalho, index=[lda_index])

CPU times: user 5.14 s, sys: 9.18 s, total: 14.3 s
Wall time: 1.8 s


In [45]:
display(pca_DMC_df)
display(lda_DMC_df)

,Média,Mediana,Mínimo,Máximo,Desv. Padrão,Sensib. média,Especif. média
PCA+DMC ['min-max'],0.694746,0.694915,0.593220,0.830508,0.046075,0.746328,0.532654
PCA+DMC ['std'],0.698814,0.694915,0.559322,0.864407,0.055693,0.760625,0.509329


,Média,Mediana,Mínimo,Máximo,Desv. Padrão,Sensib. média,Especif. média
LDA+DMC ['min-max'],0.824407,0.830508,0.728814,0.915254,0.037251,0.840639,0.773937
LDA+DMC ['std'],0.823051,0.830508,0.711864,0.915254,0.043233,0.853323,0.732730


O classificador DMC construído em cima dos dados transformados por PCA apresentou um desempenho menor do que o construído sobre os dados originais, já o DMC sobre os dados transformados por LDA apresentou uma melhora significativa em suas métricas de avaliação. Os melhores resultados para PCA/LDA+DMC se encontram abaixo:

In [46]:
display(pca_DMC_df.sort_values('Média', ascending=False).iloc[0,:].to_frame().transpose())
display(lda_DMC_df.sort_values('Média', ascending=False).iloc[0,:].to_frame().transpose())

,Média,Mediana,Mínimo,Máximo,Desv. Padrão,Sensib. média,Especif. média
PCA+DMC ['std'],0.698814,0.694915,0.559322,0.864407,0.055693,0.760625,0.509329


,Média,Mediana,Mínimo,Máximo,Desv. Padrão,Sensib. média,Especif. média
LDA+DMC ['min-max'],0.824407,0.830508,0.728814,0.915254,0.037251,0.840639,0.773937


## 8.5 PCA/LDA+CQG

Primeiro foram calculados os postos das matrizes de covariância de cada classe após as transformações exercidas pelo PCA e LDA, buscando assim estudar a invertibilidade:

In [47]:
rodadas = 100
for i in range(rodadas):
    # divisão treino/teste 
    X_train, X_test, y_train, y_test = train_test_split(features.values, 
                                                        targets.values,
                                                        test_size=0.3)
    # PCA
    pca = PCA(n_components = 3)
    pca_X = pca.fit(X_train).transform(X_train)

    # LDA
    lda = LDA(n_components = 3)
    lda.fit(X_train, y_train)
    lda_X = lda.transform(X_train)

    # Cálculo das médias
    pca_m_park = np.matmul(pca_X.T, (y_train==1)) / sum(y_train)
    lda_m_park = np.matmul(lda_X.T, (y_train==1)) / sum(y_train)
    pca_m_saud = np.matmul(pca_X.T, (y_train==0)) / (len(y_train) - sum(y_train))
    lda_m_saud = np.matmul(lda_X.T, (y_train==0)) / (len(y_train) - sum(y_train))

    # Cálculo das matrizes de covariância
    pca_C_park = np.zeros((pca_X.shape[1],pca_X.shape[1]))
    lda_C_park = np.zeros((lda_X.shape[1],lda_X.shape[1]))
    pca_C_saud = np.zeros((pca_X.shape[1],pca_X.shape[1]))
    lda_C_saud = np.zeros((lda_X.shape[1],lda_X.shape[1]))

    for j in range(len(y_train)):
        if y_train[j]: # indivíduo tem parkinson
            pca_C_park += np.matmul(np.expand_dims((pca_X[j,:]-pca_m_park), axis=1),
                                    np.expand_dims((pca_X[j,:]-pca_m_park), axis=0))
            lda_C_park += np.matmul(np.expand_dims((lda_X[j,:]-lda_m_park), axis=1),
                                    np.expand_dims((lda_X[j,:]-lda_m_park), axis=0))
        else: # indivíduo saudável
            pca_C_saud += np.matmul(np.expand_dims((pca_X[j,:]-pca_m_saud), axis=1),
                                    np.expand_dims((pca_X[j,:]-pca_m_saud), axis=0))
            lda_C_saud += np.matmul(np.expand_dims((lda_X[j,:]-lda_m_saud), axis=1),
                                    np.expand_dims((lda_X[j,:]-lda_m_saud), axis=0))

    # dividindo pelo número de elementos de cada classe
    pca_C_park/=sum(y_train)
    lda_C_park/=sum(y_train)
    pca_C_saud/=(len(y_train) - sum(y_train))
    lda_C_saud/=(len(y_train) - sum(y_train))
    
    if any([matrix_rank(pca_C_park) < 3,
           matrix_rank(pca_C_saud) < 3,
           matrix_rank(lda_C_park) < 3,
           matrix_rank(lda_C_saud) < 3]
          ):
        print("Rodada {}".format(i))
        print("Posto(PCA+C_parkinson) = {}\nPosto(PCA+C_saudavel) = {}".format(
            matrix_rank(pca_C_park), matrix_rank(pca_C_saud)))
        print("Posto(LDA+C_parkinson) = {}\nPosto(LDA+C_saudavel) = {}".format(
            matrix_rank(lda_C_park), matrix_rank(lda_C_saud)))
        break

Rodada 11
Posto(PCA+C_parkinson) = 3
Posto(PCA+C_saudavel) = 3
Posto(LDA+C_parkinson) = 2
Posto(LDA+C_saudavel) = 2


Percebe-se que não é possível passar 100 rodadas com as matrizes sendo invertíveis, sendo então necessário, para assegurar consistência na metodologia de avaliação, não utilizar o CQG.

## 8.6 PCA/LDA + Redes Neurais Artificiais

### 8.6.1 Perceptron simples (PS)

Como exemplo da classe *PerceptronSimples* em funcionamento nos dados transformados, o código abaixo apresenta o aprendizado do classificador:

In [48]:
%%time
# divisão treino/teste 
X_train, X_test, y_train, y_test = train_test_split(features.values, 
                                                    targets.values,
                                                    test_size=0.3)
# PCA
pca = PCA(n_components = 3)
pca_X_train = pca.fit(X_train).transform(X_train)
pca_X_test  = pca.transform(X_test)

# LDA
lda = LDA(n_components = 3)
lda.fit(X_train, y_train)
lda_X_train = lda.transform(X_train)
lda_X_test  = lda.transform(X_test)

# escalonar os dados
pca_X_tr_norm, pca_X_ts_norm = scale_feat(pca_X_train, pca_X_test, scaleType='min-max')
lda_X_tr_norm, lda_X_ts_norm = scale_feat(lda_X_train, lda_X_test, scaleType='min-max')


# Classificador
eta = 1e-4
nEpocas = 500
pca_ps = PerceptronSimples(3,1)
lda_ps = PerceptronSimples(3,1)
pca_ps.initParam()
lda_ps.initParam()
pca_ps.treinar(pca_X_tr_norm, y_train, pca_X_ts_norm, y_test, eta=eta, nEpocas=nEpocas)
lda_ps.treinar(lda_X_tr_norm, y_train, lda_X_ts_norm, y_test, eta=eta, nEpocas=nEpocas)

pca_ps.plotTrainHist()
lda_ps.plotTrainHist()

CPU times: user 7.31 s, sys: 677 ms, total: 7.98 s
Wall time: 7.17 s


Posteriormente o classificador foi avaliado em 100 rodadas independentes de treino/teste. Como hiper-parâmetro foi analisado o efeito to tipo de normalização dos atributos.

In [49]:
%%time

rodadas = 100

# matriz que guardará os resultados numéricos
pca_ps_data = np.zeros((len(scales), len(cabecalho)))
lda_ps_data = np.zeros((len(scales), len(cabecalho)))

for j in range(len(scales)):
    pca_acc = [0]*rodadas
    lda_acc = [0]*rodadas
    pca_especificidade = 0
    lda_especificidade = 0
    pca_sensibilidade  = 0
    lda_sensibilidade  = 0
    for i in range(rodadas):
        # divisão treino/teste 
        X_train, X_test, y_train, y_test = train_test_split(features.values, 
                                                            targets.values,
                                                            test_size=0.3)
        # PCA
        pca = PCA(n_components = 3)
        pca_X_train = pca.fit(X_train).transform(X_train)
        pca_X_test  = pca.transform(X_test)

        # LDA
        lda = LDA(n_components = 3)
        lda.fit(X_train, y_train)
        lda_X_train = lda.transform(X_train)
        lda_X_test  = lda.transform(X_test)

        # escalonar os dados
        pca_X_tr_norm, pca_X_ts_norm = scale_feat(pca_X_train, pca_X_test, scaleType=scales[j])
        lda_X_tr_norm, lda_X_ts_norm = scale_feat(lda_X_train, lda_X_test, scaleType=scales[j])
        
        # construindo o classificador
        pca_ps = PerceptronSimples(3,1)
        lda_ps = PerceptronSimples(3,1)
        pca_ps.initParam()
        lda_ps.initParam()
        
        eta=1e-4
        nEpocas=300
        pca_ps.treinar(pca_X_tr_norm, y_train, pca_X_ts_norm, y_test, eta=eta, nEpocas=nEpocas)
        lda_ps.treinar(lda_X_tr_norm, y_train, lda_X_ts_norm, y_test, eta=eta, nEpocas=nEpocas)

        # calculando as métricas de avaliação
        cm = confusion_matrix(y_test, pca_ps.prever(pca_X_ts_norm))
        total=sum(sum(cm))
        pca_acc[i] = (cm[0,0]+cm[1,1])/total
        pca_especificidade += cm[0,0]/(cm[0,0]+cm[0,1])
        pca_sensibilidade  += cm[1,1]/(cm[1,1]+cm[1,0])
        
        cm = confusion_matrix(y_test, lda_ps.prever(lda_X_ts_norm))
        total=sum(sum(cm))
        lda_acc[i] = (cm[0,0]+cm[1,1])/total
        lda_especificidade += cm[0,0]/(cm[0,0]+cm[0,1])
        lda_sensibilidade  += cm[1,1]/(cm[1,1]+cm[1,0])

    pca_especificidade/=rodadas # Valores médios
    lda_especificidade/=rodadas 
    pca_sensibilidade /=rodadas
    lda_sensibilidade /=rodadas
    
    pca_ps_data[j,:] = np.matrix([np.mean(pca_acc), np.median(pca_acc), min(pca_acc), max(pca_acc), 
                                  np.std(pca_acc), pca_sensibilidade, pca_especificidade])
    lda_ps_data[j,:] = np.matrix([np.mean(lda_acc), np.median(lda_acc), min(lda_acc), max(lda_acc), 
                                  np.std(lda_acc), lda_sensibilidade, lda_especificidade])

pca_index = ['PCA+PS [\'{}\']'.format(scale) for scale in scales]
lda_index = ['LDA+PS [\'{}\']'.format(scale) for scale in scales]
pca_ps_df = pd.DataFrame(pca_ps_data, columns=cabecalho, index=pca_index)
lda_ps_df = pd.DataFrame(lda_ps_data, columns=cabecalho, index=lda_index)

CPU times: user 13min 35s, sys: 1min 47s, total: 15min 23s
Wall time: 12min 52s


In [50]:
display(pca_ps_df)
display(lda_ps_df)

,Média,Mediana,Mínimo,Máximo,Desv. Padrão,Sensib. média,Especif. média
PCA+PS ['min-max'],0.603051,0.593220,0.372881,0.830508,0.100278,0.723693,0.240808
PCA+PS ['std'],0.733729,0.745763,0.440678,0.847458,0.083492,0.831911,0.434712


,Média,Mediana,Mínimo,Máximo,Desv. Padrão,Sensib. média,Especif. média
LDA+PS ['min-max'],0.593898,0.567797,0.355932,0.898305,0.118884,0.712798,0.231971
LDA+PS ['std'],0.819661,0.830508,0.644068,0.915254,0.055573,0.870273,0.668128


Podemos perceber que para ambas as técnicas de redução de dimensionalidade o PS teve melhor desempenho quando foi aplicada a normalização estatística. Os melhores resultado para PCA/LDA+PS foram:

In [51]:
display(pca_ps_df.sort_values('Média', ascending=False).iloc[0,:].to_frame().transpose())
display(lda_ps_df.sort_values('Média', ascending=False).iloc[0,:].to_frame().transpose())

,Média,Mediana,Mínimo,Máximo,Desv. Padrão,Sensib. média,Especif. média
PCA+PS ['std'],0.733729,0.745763,0.440678,0.847458,0.083492,0.831911,0.434712


,Média,Mediana,Mínimo,Máximo,Desv. Padrão,Sensib. média,Especif. média
LDA+PS ['std'],0.819661,0.830508,0.644068,0.915254,0.055573,0.870273,0.668128


### 8.6.2 Perceptron multicamadas (MLP)

Para exemplificar a classe *PerceptronMulticamada* em funcionamento nos dados transformados o código abaixo foi executado.

In [60]:
%%time
# divisão treino/teste 
X_train, X_test, y_train, y_test = train_test_split(features.values, 
                                                    targets.values,
                                                    test_size=0.3)
# PCA
pca = PCA(n_components=3)
pca.fit(X_train)
pca_X_train = pca.transform(X_train)
pca_X_test  = pca.transform(X_test)

# LDA
lda = LDA(n_components=3)
lda.fit(X_train, y_train)
lda_X_train = lda.transform(X_train)
lda_X_test  = lda.transform(X_test)

# escalonamento dos dados
pca_X_tr_norm, pca_X_ts_norm = scale_feat(pca_X_train, pca_X_test, scaleType='min-max')
lda_X_tr_norm, lda_X_ts_norm = scale_feat(lda_X_train, lda_X_test, scaleType='min-max')

# Construção e treino do modelo
nNeuronios = 15
eta = 1e-4
nEpocas = 100

pca_mlp = PerceptronMulticamada(3, nNeuronios, 1)
lda_mlp = PerceptronMulticamada(3, nNeuronios, 1)
pca_mlp.initParam()
lda_mlp.initParam()

pca_mlp.treinar(pca_X_tr_norm, y_train, pca_X_ts_norm, y_test, eta=eta, nEpocas=nEpocas)
lda_mlp.treinar(lda_X_tr_norm, y_train, lda_X_ts_norm, y_test, eta=eta, nEpocas=nEpocas)

pca_mlp.plotTrainHist()
lda_mlp.plotTrainHist()

CPU times: user 6.81 s, sys: 490 ms, total: 7.3 s
Wall time: 6.67 s


Posteriormente o classificador foi avaliado em 100 rodadas independentes de treino/teste. Como hiper-parâmetro só foi analisado a influência do tipo de normalização dos atributos. O número de neurônios da camada oculta, 15, foi escolhido para manter aproximadamente a mesma proporção entre dimensão da entrada e número de neurônos que a MLP implementada no tópico 7.2.

In [53]:
%%time

rodadas = 100

# matriz que guardará os resultados numéricos
pca_mlp_data = np.zeros((len(scales), len(cabecalho)))
lda_mlp_data = np.zeros((len(scales), len(cabecalho)))

for j in range(len(scales)):
    pca_acc = [0]*rodadas
    lda_acc = [0]*rodadas
    pca_especificidade = 0
    lda_especificidade = 0
    pca_sensibilidade  = 0
    lda_sensibilidade  = 0
    for i in range(rodadas):
        # divisão treino/teste 
        X_train, X_test, y_train, y_test = train_test_split(features.values, 
                                                            targets.values,
                                                            test_size=0.3)
        # PCA
        pca = PCA(n_components=3)
        pca.fit(X_train)
        pca_X_train = pca.transform(X_train)
        pca_X_test  = pca.transform(X_test)
        
        # LDA
        lda = LDA(n_components=3)
        lda.fit(X_train, y_train)
        lda_X_train = lda.transform(X_train)
        lda_X_test  = lda.transform(X_test)
        
        # escalonar os dados
        pca_X_tr_norm, pca_X_ts_norm = scale_feat(pca_X_train, pca_X_test, scaleType=scales[j])
        lda_X_tr_norm, lda_X_ts_norm = scale_feat(lda_X_train, lda_X_test, scaleType=scales[j])
        
        # construindo o classificador
        nNeuronios = 15
        eta = 1e-4
        nEpocas = 100
        
        pca_mlp = PerceptronMulticamada(3, nNeuronios, 1)
        lda_mlp = PerceptronMulticamada(3, nNeuronios, 1)
        pca_mlp.initParam()
        lda_mlp.initParam()
        pca_mlp.treinar(pca_X_tr_norm, y_train, pca_X_ts_norm, y_test, eta=eta, nEpocas=nEpocas)
        lda_mlp.treinar(lda_X_tr_norm, y_train, lda_X_ts_norm, y_test, eta=eta, nEpocas=nEpocas)

        # calculando as métricas de avaliação
        cm = confusion_matrix(y_test, pca_mlp.prever(pca_X_ts_norm)[0]>0)
        total=sum(sum(cm))
        pca_acc[i] = (cm[0,0]+cm[1,1])/total
        pca_especificidade += cm[0,0]/(cm[0,0]+cm[0,1])
        pca_sensibilidade  += cm[1,1]/(cm[1,1]+cm[1,0])
        
        cm = confusion_matrix(y_test, lda_mlp.prever(lda_X_ts_norm)[0]>0)
        total=sum(sum(cm))
        lda_acc[i] = (cm[0,0]+cm[1,1])/total
        lda_especificidade += cm[0,0]/(cm[0,0]+cm[0,1])
        lda_sensibilidade  += cm[1,1]/(cm[1,1]+cm[1,0])

    pca_especificidade/=rodadas # Valores médios
    lda_especificidade/=rodadas
    pca_sensibilidade /=rodadas
    lda_sensibilidade /=rodadas
    
    pca_mlp_data[j,:] = np.matrix([np.mean(pca_acc), np.median(pca_acc), min(pca_acc), 
                                   max(pca_acc), np.std(pca_acc), pca_sensibilidade, 
                                   pca_especificidade])
    lda_mlp_data[j,:] = np.matrix([np.mean(lda_acc), np.median(lda_acc), min(lda_acc), 
                                   max(lda_acc), np.std(lda_acc), lda_sensibilidade, 
                                   lda_especificidade])

pca_index = ['PCA+MLP [\'{}\']'.format(scale) for scale in scales]
lda_index = ['LDA+MLP [\'{}\']'.format(scale) for scale in scales]
pca_mlp_df = pd.DataFrame(pca_mlp_data, columns=cabecalho, index=pca_index)
lda_mlp_df = pd.DataFrame(lda_mlp_data, columns=cabecalho, index=lda_index)

CPU times: user 20min 45s, sys: 1min 48s, total: 22min 34s
Wall time: 20min 3s


In [54]:
display(pca_mlp_df)
display(lda_mlp_df)

,Média,Mediana,Mínimo,Máximo,Desv. Padrão,Sensib. média,Especif. média
PCA+MLP ['min-max'],0.749492,0.745763,0.644068,0.864407,0.044495,1.000000,0.000000
PCA+MLP ['std'],0.750847,0.745763,0.627119,0.898305,0.057051,0.977508,0.069932


,Média,Mediana,Mínimo,Máximo,Desv. Padrão,Sensib. média,Especif. média
LDA+MLP ['min-max'],0.749492,0.745763,0.644068,0.864407,0.044495,1.000000,0.000000
LDA+MLP ['std'],0.753729,0.754237,0.559322,0.898305,0.059658,0.975449,0.087055


O desempenho do MLP para os dados transformados e os dados originais foi praticamente o mesmo, também não houve diferença significativa entre PCA e LDA. Os melhores resultados de PCA/LDA+MLP foram:

In [55]:
display(pca_mlp_df.sort_values('Média', ascending=False).iloc[0,:].to_frame().transpose())
display(lda_mlp_df.sort_values('Média', ascending=False).iloc[0,:].to_frame().transpose())

,Média,Mediana,Mínimo,Máximo,Desv. Padrão,Sensib. média,Especif. média
PCA+MLP ['std'],0.750847,0.745763,0.627119,0.898305,0.057051,0.977508,0.069932


,Média,Mediana,Mínimo,Máximo,Desv. Padrão,Sensib. média,Especif. média
LDA+MLP ['std'],0.753729,0.754237,0.559322,0.898305,0.059658,0.975449,0.087055


# 9. Discussão dos resultados

Após todas as simulações realizadas a tabela a seguir foi consolidada para reunir os melhores resultados de cada classificador com ou sem técnicas de redução de dimensionalidade:

In [59]:
resultado = pd.concat([
    df_knn.sort_values('Média', ascending=False).iloc[0,:].to_frame().transpose(),
    reg_df.sort_values('Média', ascending=False).iloc[0,:].to_frame().transpose(),
    log_df.sort_values('Média', ascending=False).iloc[0,:].to_frame().transpose(),
    DMC_df.sort_values('Média', ascending=False).iloc[0,:].to_frame().transpose(),
    CQG_df.sort_values('Média', ascending=False).iloc[0,:].to_frame().transpose(),
    ps_df.sort_values('Média', ascending=False).iloc[0,:].to_frame().transpose(),
    mlp_df.sort_values('Média', ascending=False).iloc[0,:].to_frame().transpose(),
    
    df_knn_pca.sort_values('Média', ascending=False).iloc[0,:].to_frame().transpose(),
    pca_reg_df.sort_values('Média', ascending=False).iloc[0,:].to_frame().transpose(),
    pca_reg_log_df.sort_values('Média', ascending=False).iloc[0,:].to_frame().transpose(),
    pca_DMC_df.sort_values('Média', ascending=False).iloc[0,:].to_frame().transpose(),
    pca_ps_df.sort_values('Média', ascending=False).iloc[0,:].to_frame().transpose(),
    pca_mlp_df.sort_values('Média', ascending=False).iloc[0,:].to_frame().transpose(),

    df_knn_lda.sort_values('Média', ascending=False).iloc[0,:].to_frame().transpose(),
    lda_reg_df.sort_values('Média', ascending=False).iloc[0,:].to_frame().transpose(),
    lda_reg_log_df.sort_values('Média', ascending=False).iloc[0,:].to_frame().transpose(),
    lda_DMC_df.sort_values('Média', ascending=False).iloc[0,:].to_frame().transpose(),
    lda_ps_df.sort_values('Média', ascending=False).iloc[0,:].to_frame().transpose(),
    lda_mlp_df.sort_values('Média', ascending=False).iloc[0,:].to_frame().transpose(),
])

resultado

,Média,Mediana,Mínimo,Máximo,Desv. Padrão,Sensib. média,Especif. média
"(1-NN ['min-max'],)",0.938644,0.949153,0.864407,1.000000,0.031140,0.954675,0.893021
Reg. Linear ['min-max'],0.863898,0.864407,0.762712,0.966102,0.042370,0.944628,0.622252
"(Reg. log. [std / l2 / 10],)",0.855254,0.864407,0.711864,0.932203,0.039707,0.920512,0.657771
"(DMC ['min-max'],)",0.756271,0.762712,0.627119,0.847458,0.044119,0.730282,0.832183
"(CQG variante 3 [$\lambda$ = 0.111],)",0.869831,0.864407,0.762712,0.983051,0.043535,0.920011,0.718591
PS ['min-max'],0.787797,0.788136,0.661017,0.898305,0.043575,0.863032,0.572168
MLP ['std'],0.780000,0.779661,0.627119,0.881356,0.048118,0.987775,0.182239
"(PCA+5-NN ['min-max'],)",0.830169,0.830508,0.711864,0.949153,0.044132,0.947021,0.478715
PCA+Reg. Linear ['min-max'],0.817966,0.813559,0.711864,0.915254,0.043187,0.965156,0.382790
"(PCA+Reg. Log. [std / l2 / 10000],)",0.824237,0.830508,0.661017,0.915254,0.049103,0.959592,0.399696


De todos os classificadores utilizados o que teve melhor desempenho foi o 1-NN com normalização do tipo *'min-max'*. Sua taxa de acerto média de 94,3% vence com grande margem do segundo colocado, LDA+Reg. Logística com taxa  de acerto média de 87,6%. Essa foi a grande surpresa deste trabalho, o classificador mais simples apresentou o melhor desempenho, desempenho esse muito superior aos outros modelos, até os mais sofisticados como o MLP. Esse resultado reforça a ideia de que na área de *Machine Learning* se faz necessário a experimentação, ao iniciar o estudo de um novo conjunto de dados é preciso aplicar diversos modelos, evitando uma ideia viezada do "poder" ou "capacidade" de certos modelos mais simples. Outro ponto que deve ser ressaltado é que o MLP, possivelmente o modelo mais complexo utilizado, apresentou um desempenho medíocre, reforçando a ideia não existe *one model to rule them all*.

De forma geral o desempenho dos modelos piorou com a redução de dimensionalidade fornecida pelo PCA e LDA; excluindo-se alguns casos especiais com o DMC e PS com LDA. A queda em performance era esperada porque perda de informação é prevista com a projeção dos dados num subespaço menor, mas como vantagens podemos citar:
- A redução de 22 para 3 atributos diminui o peso computacional, tanto em processamento quanto memória, para construir/treinar os modelos, permitindo um trabalho mais ágil e mais tempo disponível para a otimização dos hiper-parâmetros;
- A diminuição das exigência computacionais propicia a implementação dos modelos em sistemas embarcados, que além de memória e processamento reduzidos tem que se preocupar com o consumo de energia;
- Alguns modelos tiveram seu desempenho mantido e até melhorados com a redução da dimensionalidade, o que torna o PCA/LDA nesse caso um passo obrigatório na utilização desses classificadores para o conjunto de dados estudado.

Das duas técnicas de redução de dimensionalidade o PCA foi a que se mostrou menos atrativa, de forma geral a queda de desempenho não foi negligenciável, talvez aceitável para o MLP. Já o LDA, em muitos casos, conservou e até melhorou o desempenho, com destaques para o DMC que aumentou a taxa de acerto média em 7% e o PS em 2%.

O k-NN foi o melhor classificador entre os aplicados nos dados originais e nos dados transformados por PCA, já nos transformados por LDA a Regressão Logística venceu o k-NN por uma pequena margem, podendo essa ter sido concebida pela aleatoriedade das separações entre treino/teste exercidas nas 100 rodadas de avaliação.

In [ ]:
# função para transformar pandas datrafame para tabela no latex
def print_df(df):
    print(
        "\\begin{table}[h!]\n"
        "    \captionsetup{width=16cm}%ATENÇÃO: Ajuste a largura do título\n"
        "    \Caption{\label{tab:nome_tabela} Descrição da tabela}\n"
        "    \\begin{adjustbox}{width=1\\textwidth}\n"
        "    \small\n"
        +df.to_latex()+
        "    \end{adjustbox}\n"
        "    \Fonte{O autor.}\n"
        "\end{table}")